In [146]:
# !pip install natsort
import pandas as pd
from tqdm import tqdm
from natsort import natsorted
import numpy as np
import cv2
from PIL import Image
from os.path import join as opj
import random

df = pd.read_csv('train.csv',usecols = [1,4])


# Mix

In [147]:
df_ = df[df['label'].str.contains('good')] 
df_bad = df[~df['label'].str.contains('good')] 

In [148]:
uni = df_['label'].unique()
df_good = pd.DataFrame(columns = ['file_name', 'label'])
for c, lab in enumerate(uni):
  a= df_[df_['label'].str.contains(lab)].head(10)
  df_good= pd.concat([df_good,a])

In [149]:
def mixup(img1_path,img2_path,save_name):
  alpha = 0.3 
  img1 = cv2.imread(img1_path)
  img2 = cv2.imread(img2_path)

  blended = img1 * alpha + img2 * (1-alpha)
  blended = blended.astype(np.uint8)
  dst = cv2.addWeighted(img1, alpha, img2, (1-alpha), 0)
  cv2.imwrite('C://Users/DM_lab/Music/Untitled Folder/mixup/'+save_name, dst)

In [150]:
df_good['label'].value_counts()
df_mixup = pd.DataFrame(columns = ['file_name', 'label'])
idx = 0
for i,g_row in df_good.iterrows():
  for j,b_row in df_bad.iterrows():
    g_label = g_row['label'].split('-')[0]
    b_label = b_row['label'].split('-')[0]
    if g_label == b_label:
      a = b_row['label']
      save_name = f'3{idx}.png'
      img1_path ='train/'+ g_row['file_name']
      img2_path ='train/'+ b_row['file_name']
      mixup(img1_path,img2_path,save_name)
      idx+=1
      df_mixup.loc[len(df_mixup)] = [save_name] +  [b_row['label']]

In [161]:
aug_df = df.copy()

In [162]:
def get_random_crop(image, crop_height, crop_width):

    max_x = image.shape[1] - crop_width
    max_y = image.shape[0] - crop_height

    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)

    crop = image[y: y + crop_height, x: x + crop_width]

    return crop
oslabel = list(aug_df['label'].unique())

idx = 0

for label in tqdm(oslabel):
    if 'good' not in label:
        
        one_sample = aug_df[aug_df['label'] == label].reset_index(drop=True)
        images_list = natsorted(one_sample['file_name'])
        for _, image_name in enumerate(images_list):
            image = np.array(Image.open(opj('aug', image_name)).convert('RGB'))
            aug_img = cv2.resize(image, dsize=(1024, 1024))
            aug_img = get_random_crop(aug_img, 900, 900)
            aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
            save_name = f'2{idx}.png'
            idx += 1
            cv2.imwrite('C://Users/DM_lab/Music/Untitled Folder/aug/'+save_name, aug_img)
            aug_df.loc[len(aug_df)] = [save_name] + one_sample.iloc[0][1:].values.tolist()

100%|██████████| 88/88 [00:24<00:00,  3.62it/s]


In [163]:
def rotation(img, angle):
    angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h)) 
    return img

for label in tqdm(oslabel):
    if 'good' not in label:
        one_sample = aug_df[aug_df['label'] == label].reset_index(drop=True)
        images_list = natsorted(one_sample['file_name'])
        for _, image_name in enumerate(images_list):
            image = np.array(Image.open(opj('aug', image_name)).convert('RGB'))
            
            aug_img = rotation(image, 30)  
            aug_img = cv2.resize(aug_img, dsize=(1024, 1024))
            aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
            save_name = f'2{idx}.png'
            idx += 1
            cv2.imwrite('C://Users/DM_lab/Music/Untitled Folder/aug/'+save_name, aug_img)
            aug_df.loc[len(aug_df)] = [save_name] + one_sample.iloc[0][1:].values.tolist() 

100%|██████████| 88/88 [00:51<00:00,  1.72it/s]


In [164]:
for label in tqdm(oslabel):
    if 'good' not in label:
        one_sample = aug_df[aug_df['label'] == label].reset_index(drop=True)
        images_list = natsorted(one_sample['file_name'])
        for _, image_name in enumerate(images_list):
            image = np.array(Image.open(opj('aug', image_name)).convert('RGB'))
            
            aug_img = cv2.flip(image, 1)
            aug_img = cv2.resize(aug_img, dsize=(1024, 1024))
            aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
            save_name = f'2{idx}.png'
            idx += 1
            cv2.imwrite('C://Users/DM_lab/Music/Untitled Folder/aug/'+save_name, aug_img)
            aug_df.loc[len(aug_df)] = [save_name] + one_sample.iloc[0][1:].values.tolist()   

100%|██████████| 88/88 [01:44<00:00,  1.19s/it]


In [102]:
train_labels = train_df["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]
train_df

,file_name,label
0,10000.png,transistor-good
1,10001.png,capsule-good
2,10002.png,transistor-good
3,10003.png,wood-good
4,10004.png,bottle-good
...,...,...
2995,12995.png,screw-good
2996,12996.png,wood-good
2997,12997.png,zipper-good
2998,12998.png,tile-oil


In [165]:
aug_df__ = aug_df.copy()

In [166]:
aug_df__=aug_df__.drop_duplicates(['file_name'])

In [167]:
train_df_add_data = pd.concat([df_mixup,aug_df__])

In [168]:
train_df_add_data.to_csv('train_df_add_data.csv')
aug_df__.to_csv('train_df_aug.csv')

In [143]:
train_df_add_data

,file_name,label
0,transistor-bent_lead_0_MixUp.png,transistor-bent_lead
1,transistor-damaged_case_1_MixUp.png,transistor-damaged_case
2,transistor-misplaced_2_MixUp.png,transistor-misplaced
3,transistor-misplaced_3_MixUp.png,transistor-misplaced
4,transistor-cut_lead_4_MixUp.png,transistor-cut_lead
...,...,...
6163,wood-liquid_7_flip1.png,wood-liquid
6164,wood-liquid_8_flip1.png,wood-liquid
6165,wood-liquid_9_flip1.png,wood-liquid
6166,wood-liquid_10_flip1.png,wood-liquid
